In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets 
import matplotlib.pyplot as plt

In [3]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
plotly.tools.set_credentials_file(username="rprahlad", api_key="rUy38m0nRprXxPC5fWpP")
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)
from ipywidgets import widgets
from IPython.display import display, clear_output, Image
from plotly.widgets import GraphWidget

<IPython.core.display.Javascript object>

# Welcome to the Interactive UI Notebook!
We have 3 different types of visualizations you can play with. 
Simply scroll down to each one and enter your inputs!
Please dont change any of the code, and be patient as some of them can take a couple minutes to run/update. 

### Visualizations 
#### 1. [Number of Alumni in Top Companies](#one)<br>
#### 2. [Number of Promotions By University](#two)<br>
#### 3. [Number of Hires in Top Tech Jobs by Universities by Year](#three)<br>

In [55]:
non_berk_edu = pd.read_csv("non_berk_edu.csv", low_memory=False)

In [56]:
tech_berk = pd.read_csv("tech_berk.csv", low_memory=False)
tech_non_berk = pd.read_csv("tech_non_berk.csv", low_memory=False)

In [6]:
top_companies = ['Google','Facebook','Microsoft','Adobe','Apple','Apple Inc','Uber','Airbnb','Lyft',
                 'Amazon','LinkedIn','eBay','IBM','Salesforce','Intel','Oracle','PayPal','Cisco Systems',
                 'Cisco','Genentech','SAP','Nvidia','Intuit','VMWare','Tesla','Qualcomm Inc','Hewlett Packard',
                 'HP','Hewlett Packard Enterprise','Oracle Corporation','Oracle America Inc.']

top_companies = list(tech_berk[tech_berk['Company'].str.contains("Google|Nvidia|Intuit|Oracle|Cisco|PayPal|Paypal|Salesforce|LinkedIn|Linkedin|Qualcomm|Amazon|Microsoft|IBM|Airbnb|Uber|Facebook|Lyft|Tesla|Genentech|VMWare")]['Company'].unique())
top_companies.append('Hewlett Packard')
top_companies.append('HP')
top_companies.append('Intel')
top_companies.append('Apple')
top_companies.append('Apple Inc')

In [7]:
def top_comp_counts(prop_table, group1, group2):
    company1 = []
    for i in prop_table['Person']:
        count = 0
        personcomp = group1[group1['ID'] == i]['Company'].unique()
        for i in personcomp:
            if i in top_companies:
                count += 1
        company1.append(count)
    
    company2 = []
    for i in prop_table['Closest Match']:
        count = 0
        personcomp = group2[group2['ID'] == i]['Company'].unique()
        for i in personcomp:
            if i in top_companies:
                count += 1
        company2.append(count)
    
    prop_table["Company 1"] = company1
    prop_table['Company 2'] = company2
    
    return prop_table 

In [8]:
import operator
features = ['Skill_1', 'Skill_1_Weight', 'Skill_2', 'Skill_2_Weight', 
            'Education', 'Company', 'Majors', 'Role', 'Elite_Flag']

def knn(group1, group2):
    #gets k most similar people to group1 from group2 based on calculated metric
    
    features = ['ID', 'Skill_1', 'Skill_1_Weight', 'Skill_2', 'Skill_2_Weight', 
            'Education','Majors', 'Role', 'Elite_Flag']
    people = []
    matches = []
    
    group1 = group1.groupby("ID", as_index=False).last() #only keeps last instance of a person
    group2 = group2.groupby("ID", as_index=False).last() 
    
    #group2 = group2.sample(len(group1))
    group1.reset_index(drop = True, inplace = True)
    group2.reset_index(drop = True, inplace = True)
    
    for p in range(len(group1)):
        person_id = group1['ID'][p]
        people.append(person_id)
        #dict mapping each person in group2 to a "score" of similarity to person p from group1
        scores = {}
        for p2 in range(len(group2)):
            #make sure both people have same education
            if group2['Education'][p2] == group1['Education'][p]: 
                metric = 0
                their_id = group2['ID'][p2]

                if group2['Skill_1'][p2] == group1['Skill_1'][p]: #skill 1 , weight
                    metric += 5
                    metric += 1/abs(group2['Skill_1_Weight'][p2] - group1['Skill_1_Weight'][p])

                if group2['Skill_2'][p2] == group1['Skill_2'][p]: #skill 2, weight 
                    metric += 5
                    metric += 1/abs(group2['Skill_2_Weight'][p2] - group1['Skill_2_Weight'][p])

                if group2['Elite_Flag'][p2] == group1['Elite_Flag'][p]: #Elite Flag bool
                    metric += 3

                if group2['Majors'][p2] == group1['Majors'][p]: #major
                    metric += 3
                    
                if group2['Role'][p2] == group1['Role'][p]: #Role
                    metric += 3

                scores[their_id] = metric
        
        if person_id in scores.keys():
            scores.pop(person_id)
        closest_person = max(scores.items(), key=operator.itemgetter(1))[0]
        matches.append(closest_person)        
    
    table = {'Person': people , 'Closest Match': matches }
    return pd.DataFrame(table)
      

In [9]:
choices = non_berk_edu["Company"].unique()
len(choices)

171510

## Number of Alumni in Top Companies <a id='one'></a>

In [47]:
textbox = widgets.Text(description="University:", options = list(non_berk_edu['Company'].unique()),
                        value= "University")

out = widgets.Output()
trace1 = go.Bar(x = ["Berkeley", textbox.value], y=[0,0]) 
g = go.FigureWidget(data=[trace1], 
                   layout = go.Layout(
                   title=dict(
                   text= "Number of Alumni Working at Bay Area Top Tech Companies (Out of a Sample of 1000)")))
button = widgets.Button(description='Submit')

In [48]:
def validate():
    if textbox.value in non_berk_edu['Company'].unique():
        valid.value = True
        return True
    
    else:
        valid.value = False
        return False

    
def response(change):
    if validate():
        non_berk_edu = pd.read_csv('non_berk_edu.csv')
        tech_non_berk = pd.read_csv('tech_non_berk.csv') 
        non_berk_edu = non_berk_edu[non_berk_edu["Company"].isna() == False]
        
        un1_edu = non_berk_edu[(non_berk_edu['Company'].str.contains(str(textbox.value)))]
        un1_edu_cleaned = un1_edu[((un1_edu['Skill_1'] != str(-1)) & (un1_edu['Skill_2'] != str(-1)))]
        
        ids = un1_edu_cleaned["ID"].unique()
        tech_un1 = tech_non_berk.loc[tech_non_berk["ID"].isin(ids)]
        prop_table = knn(tech_berk[0:1000], tech_un1[0:1000])
        comp_counts = top_comp_counts(prop_table, tech_berk, tech_un1)  

        x1 = sum(comp_counts["Company 1"])
        x2 = sum(comp_counts["Company 2"])
        with g.batch_update():
            g.data[0].y = [x1, x2]
            
        
        
valid = widgets.Valid(valid=True)        
textbox.observe(response, names="value")
button.on_click(response)       


In [49]:
container = widgets.HBox(children=[textbox, button, valid]) 
widgets.VBox(children=[container, g, out])
display(container)
display(g) 


FigureWidget({
    'data': [{'type': 'bar', 'uid': 'd3ac20b1-ccd7-49c5-93fc-c735c32df00d', 'x': ['Berkeley', '…

## Number of Promotions By University <a id='two'></a>

In [57]:
def promotion_counts(school):
    
    non_berk_edu = pd.read_csv('non_berk_edu.csv', low_memory=False)
    non_berk_edu_cleaned = non_berk_edu[((non_berk_edu['Skill_1'] != str(-1))
                            & (non_berk_edu['Skill_2'] != str(-1)))]
    
    non_berk_edu_cleaned = non_berk_edu_cleaned[non_berk_edu_cleaned["Company"].isna() == False]
    
    if any(non_berk_edu_cleaned['Company'].str.contains(school)):
        
        tech_non_berk = pd.read_csv("tech_non_berk.csv", low_memory=False)
        tech_berk = pd.read_csv("tech_berk.csv", low_memory=False)
        berk_ids = np.random.choice(tech_berk['ID'].unique(), 1000, replace=False)
        tech_berk = tech_berk.loc[tech_berk["ID"].isin(berk_ids)]
        
        un1_edu = non_berk_edu_cleaned[(non_berk_edu_cleaned['Company'].str.contains(school))]
        un1_edu_cleaned = un1_edu[((un1_edu['Skill_1'] != str(-1)) & (un1_edu['Skill_2'] != str(-1)))]

        ids = np.random.choice(un1_edu_cleaned['ID'].unique(), 1000, replace=False)
        tech_un1 = tech_non_berk.loc[tech_non_berk["ID"].isin(ids)]
        
        total_counts = tech_un1.groupby("ID").agg({"Company": pd.Series.count})
        unique_counts = tech_un1.groupby("ID").agg({"Company": pd.Series.nunique}) 
        promo_ct = (total_counts - unique_counts)['Company'].sum()
        
        total_countsb = tech_berk.groupby("ID").agg({"Company": pd.Series.count})
        unique_countsb = tech_berk.groupby("ID").agg({"Company": pd.Series.nunique}) 
        promo_ctb = (total_countsb - unique_countsb)['Company'].sum()
        
        #Plotting (don't need rn)
        ind = np.arange(500)
        fig, ax = plt.subplots(figsize = (10,7))
        nums = [promo_ctb, promo_ct]
        ax.bar(['UC Berkeley', school],nums, color=["gold","darkblue"])
        for index,data in enumerate(nums):
            plt.text(x=index , y =data+1 , s=f"{data}" , fontdict=dict(fontsize=12))
        plt.title("Counts of Promotions of Berkeley Alumni vs " + school + " Alumni")
        plt.xlabel("University")
        plt.ylabel("Count")
        plt.show()
    else:
        print("This dataset does not contain data for this university")



In [58]:
textbox = widgets.Text(description = "University:", options = list(non_berk_edu['Company'].unique()),
                        value= "University")

out = widgets.Output()

trace1 = go.Bar(x = ["Berkeley", textbox.value], y=[0,0]) 
g = go.FigureWidget(data=[trace1], 
                   layout = go.Layout(
                   title=dict(
                   text= "Number of Promotions of Alumni Working at Bay Area Top Tech Companies (Out of a Sample of 1000)")))
button = widgets.Button(description='Submit')


In [59]:
def validate():
    # print "validating"
    if textbox.value in non_berk_edu['Company'].unique():
        valid.value = True
        return True
    else:
        valid.value = False 
        return False


def response(change):
    if validate():
        
        non_berk_edu = pd.read_csv('non_berk_edu.csv', low_memory=False)
        non_berk_edu_cleaned = non_berk_edu[((non_berk_edu['Skill_1'] != str(-1))
                            & (non_berk_edu['Skill_2'] != str(-1)))]
    
        non_berk_edu_cleaned = non_berk_edu_cleaned[non_berk_edu_cleaned["Company"].isna() == False]
        
        tech_non_berk = pd.read_csv("tech_non_berk.csv")
        tech_berk = pd.read_csv("tech_berk.csv")
        berk_ids = np.random.choice(tech_berk['ID'].unique(), 1000, replace=False)
        tech_berk = tech_berk.loc[tech_berk["ID"].isin(berk_ids)]
        
        un1_edu = non_berk_edu_cleaned[(non_berk_edu_cleaned['Company'].str.contains(textbox.value))]
        un1_edu_cleaned = un1_edu[((un1_edu['Skill_1'] != str(-1)) & (un1_edu['Skill_2'] != str(-1)))]

        ids = np.random.choice(un1_edu_cleaned['ID'].unique(), 1000, replace=False)
        tech_un1 = tech_non_berk.loc[tech_non_berk["ID"].isin(ids)]
        
        total_counts = tech_un1.groupby("ID").agg({"Company": pd.Series.count})
        unique_counts = tech_un1.groupby("ID").agg({"Company": pd.Series.nunique}) 
        promo_ct = (total_counts - unique_counts)['Company'].sum()
        
        total_countsb = tech_berk.groupby("ID").agg({"Company": pd.Series.count})
        unique_countsb = tech_berk.groupby("ID").agg({"Company": pd.Series.nunique}) 
        promo_ctb = (total_countsb - unique_countsb)['Company'].sum()
        

        
        x1 = promo_ctb
        x2 = promo_ct
        with g.batch_update():
            g.data[0].y = [x1, x2]
            #g.data[1]
        
        
valid = widgets.Valid(valid=True)        
textbox.observe(response, names="value")
button.on_click(response)       


In [60]:
container = widgets.HBox(children=[textbox, message, button, valid]) 
widgets.VBox(children=[container, g, out])
display(container)
display(g)


FigureWidget({
    'data': [{'type': 'bar', 'uid': '4888b3ba-bcf6-48e0-aa77-d3e674a5bba9', 'x': ['Berkeley', '…

## Number of Hires in Top Tech Jobs by Universities by Year <a id='three'></a> 

In [51]:
w1 = widgets.Checkbox(
    value=False,
    description='UC Berkeley',
    disabled=False
)

w2 = widgets.Checkbox(
    value=False,
    description='SCU',
    disabled=False
)

w3 = widgets.Checkbox(
    value=False,
    description='SJSU',
    disabled=False
)

w4 = widgets.Checkbox(
    value=False,
    description='UC Davis',
    disabled=False
)

w5 = widgets.Checkbox(
    value=False,
    description='Stanford',
    disabled=False
)

dropdown = widgets.Dropdown(
    options=[str(i) for i in np.arange(2000, 2019)],
    value='2000',
    description='Year',
    disabled=False,
)

out = widgets.Output()
message = widgets.HTML(value="")

In [52]:
#helper to get count of hires given university and year
berk_edu = pd.read_csv("berk_edu.csv")

def get_hires(uni, year, month):
    if uni == "UC Berkeley":
        ids = berk_edu["ID"].unique()
        jobs = tech_berk[tech_berk["ID"].isin(ids)]
        jobs = jobs[jobs["Start_Date"].str[0:4] == str(year)]  #filter by year
        jobs = jobs[jobs["Start_Date"].str[5:7] == month] #filter by month
        top_jobs = jobs[jobs["Company"].isin(top_companies)] #filter by top_companies
    else: 
        
        school = non_berk_edu[non_berk_edu["Company"] == uni] #filter by school
        ids = school["ID"].unique()
        jobs = tech_non_berk[tech_non_berk["ID"].isin(ids)]
        jobs = jobs[jobs["Start_Date"].str[0:4] == str(year)]  #filter by year
        jobs = jobs[jobs["Start_Date"].str[5:7]== month]
        top_jobs = jobs[jobs["Company"].isin(top_companies)] #filter by top_companies

    return len(top_jobs)        

In [53]:
#user is able to select Universities, Year

months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
trace0 = go.Scatter(
x = ["Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"],
y = [] , 
mode = 'lines',
name = 'UC Berkeley')

trace1 = go.Scatter(
x = ["Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"],
y = [] , 
mode = 'lines',
name = 'SCU')

trace2 = go.Scatter(
x = ["Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"],
y = [] , 
mode = 'lines',
name = 'SJSU')

trace3 = go.Scatter(
x = ["Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"],
y = [] , 
mode = 'lines',
name = 'UC Davis')

trace4 = go.Scatter(
x = ["Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"],
y = [] , 
mode = 'lines',
name = 'Stanford')


g3 = go.FigureWidget(data=[trace0, trace1, trace2, trace3, trace4], 
                   layout = go.Layout(
                   title=dict(
                   text= "Number of Top Bay Area Company Hires from Different Universities by Month" )))

def response(change):
    
    with g3.batch_update():
            if w1.value:
                g3.data[0].y = [get_hires("UC Berkeley", int(dropdown.value), i) for i in months]
            else:
                g3.data[0].y = []
            
            if w2.value:
                g3.data[1].y = [get_hires("Santa Clara University", int(dropdown.value), i) for i in months]
            else:
                g3.data[1].y = []
            if w3.value:
                g3.data[2].y = [get_hires("San Jose State University", int(dropdown.value), i) for i in months]
            else:
                g3.data[2].y = []
            if w4.value:
                g3.data[3].y = [get_hires("UC Davis", int(dropdown.value), i) for i in months]
            else:
                g3.data[3].y = []
            if w5.value:
                g3.data[4].y = [get_hires("Stanford University", int(dropdown.value), i) for i in months]
            else:
                g3.data[4].y = []
    

dropdown.observe(response, names="value")        

In [54]:
container = widgets.HBox(children=[dropdown, message, w1, w5, w3, w4, w2]) #button, valid,
widgets.VBox(children=[container, g3, out])
display(container)
display(g3)


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'UC Berkeley',
              'type': 'scat…